# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4868, uptime 0:00:34
mini_beamline                    RUNNING   pid 4869, uptime 0:00:34
random_walk                      RUNNING   pid 4870, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4871, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4872, uptime 0:00:34
simple                           RUNNING   pid 4873, uptime 0:00:34
thermo_sim                       RUNNING   pid 4874, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4875, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-06-17 20:19:35
Persistent Unique Scan ID: '9beb7580-4117-44b9-95e7-d2d8a5f2b2c5'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:19:35.6 |     -1.000 |      94956 |
|         2 | 20:19:35.6 |     -0.500 |      95677 |
|         3 | 20:19:35.7 |      0.000 |      95847 |
|         4 | 20:19:35.7 |      0.500 |      95331 |


|         5 | 20:19:35.8 |      1.000 |      94484 |


+-----------+------------+------------+------------+
generator scan ['9beb7580'] (scan num: 3)





('9beb7580-4117-44b9-95e7-d2d8a5f2b2c5',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/9beb7580-4117-44b9-95e7-d2d8a5f2b2c5-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-06-17 20:19:36
Persistent Unique Scan ID: 'e6d595d2-3edf-43a6-86c9-888d2e3c4ec7'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:19:36.2 |          -1 | 1053571682 |


|         2 | 20:19:36.4 |          -0 | 1054662455 |


|         3 | 20:19:36.5 |           0 | 1057511129 |


|         4 | 20:19:36.6 |           0 | 1061159841 |


|         5 | 20:19:36.8 |           1 | 1067570411 |


+-----------+------------+-------------+------------+
generator scan ['e6d595d2'] (scan num: 4)





('e6d595d2-3edf-43a6-86c9-888d2e3c4ec7',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-06-17 20:19:37
Persistent Unique Scan ID: '37ddb494-5c7b-4d47-9ec9-0920564f779b'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:19:37.3 | 1090369670 |


|         2 | 20:19:38.2 | 1131687453 |


|         3 | 20:19:39.2 | 1089928528 |


...

|        28 | 20:20:04.2 | 1053232953 |


|        29 | 20:20:05.2 | 1097298812 |


|        30 | 20:20:06.3 | 1131390706 |


+-----------+------------+------------+
generator count ['37ddb494'] (scan num: 5)





('37ddb494-5c7b-4d47-9ec9-0920564f779b',)